<a href="https://colab.research.google.com/github/AnikethDandu/traffic-sign-classification/blob/main/TrafficSignClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Convolutional Neural Network class

In [ ]:
import torch.nn as nn

class ConvNet(nn.Module):
  # Assume input image is 32x32x3
  self.PADDING_SIZE = 1
  self.KERNEL_SIZE = 3
  self.STRIDE = 1
  def __init__(self):
    super.__init__()
    self.conv1 = nn.Conv2d(3, 32, 
                           kernel_size=self.KERNEL_SIZE, 
                           stride=self.STRIDE, 
                           padding=self.PADDING_SIZE)
    self.conv2 = nn.Conv2d(32, 64, 
                           kernel_size=self.KERNEL_SIZE, 
                           stride=self.STRIDE, 
                           padding=self.PADDING_SIZE)
    self.conv3 = nn.Conv2d(64, 128, 
                           kernel_size=self.KERNEL_SIZE, 
                           stride=self.STRIDE, 
                           padding=self.PADDING_SIZE)
    self.conv4 = nn.Conv2d(128, 256, 
                           kernel_size=self.KERNEL_SIZE, 
                           stride=self.STRIDE, 
                           padding=self.PADDING_SIZE)
    